## Начало

> Код основан на репозитории [tinkoff-ai github \[1\]](https://github.com/tinkoff-ai/pycon-chit-chat)

### Для информации
- Папка для работы - `MyDrive/colab_mount`
- Папка, где лежит `json` файл чата - `MyDrive/colab_mount/data`
- Папка, для сохранения модели - `MyDrive/colab_mount/timoha`

Включаем виджеты, маунтим [google drive](https://drive.google.com), устанавливаем константы

In [ ]:
from google.colab import output, drive

output.enable_custom_widget_manager()
drive.mount("/content/drive", force_remount=True)

data_path = "/content/drive/MyDrive/colab_mount/data"
model_output_path = "/content/drive/MyDrive/colab_mount/timoha"

Проверяем, что все работает

In [ ]:
!ls /content/drive/MyDrive/colab_mount

Качаем `requirements`

In [ ]:
!pip install -r /content/drive/MyDrive/colab_mount/dev.requirements.txt

## Предобработка данных

После скачивания данных, преобразуем их при помощи данного нам скрипта

In [ ]:
!python3 /content/drive/MyDrive/colab_mount/exporter.py --tg-history-path '/content/drive/MyDrive/colab_mount/data/result.json' --output-path '/content/drive/MyDrive/colab_mount/data/result.csv'

Взглянем на данные

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(f"{data_path}/result.csv")
df.head(10)

Имеем цепочку `context_3` - `context_2` - `context_1` - `response`, где
- `context_2` - это ответ на `context_3`
- `context_1` - это ответ на `context_2`
- `response` - это ответ на `context_1`.

Тренировочными данными можно выбрать все такие диалоги, при условии, что хотя бы `context_1` будет не `NaN` (иначе не получится провалидировать ответ модели)

Я подобрал [чат](https://t.me/+s7HAQFTWTGAwMmUy), который +- модерировался, чтобы избежать токсичных высказываний, матов и прочего. Однако специфика чата (как мы увидим вдальнейшем) сильно повлияет на ответы модели.

## Загрузка и обработка данных

Удалим все данные без `context_1`

In [ ]:
df_with_context_1 = df.dropna(subset=["context_1"])
df_with_context_1.head(10)

In [ ]:
len(df_with_context_1)

Чистим данные от ссылок при помощи [`regex`](https://stackoverflow.com/questions/3809401/what-is-a-good-regular-expression-to-match-a-url)

In [ ]:
link_regex = "(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"
df_clean = df_with_context_1[
    ~df_with_context_1.apply(lambda x: x.str.contains(link_regex)).any(axis=1)
]
len(df_clean)

Для перевода данных в удобный для тренировки формат, воспользуемся библиотекой `datasets`

In [ ]:
from datasets import Dataset

In [ ]:
ds = Dataset.from_pandas(df_clean).train_test_split(test_size=0.2)
ds

Проверим, что ничего не поломалось

In [ ]:
print(ds["train"][0])
print(ds["test"][0])

По карточке модели на [huggingface](https://huggingface.co/tinkoff-ai/ruDialoGPT-medium) можно понять, что для модели надо добавить специальные токены `@@ПЕРВЫЙ@@` и `@@ВТОРОЙ@@` для отличия собеседников. Напишем для этого пару функций:

In [ ]:
first = " @@ПЕРВЫЙ@@ "
second = " @@ВТОРОЙ@@ "
cols_ordered = [
    "context_3",
    "context_2",
    "context_1",
    "response",
]


def add_tokens(conversation: list[str]) -> str:
    if not conversation:
        return f"{second} "

    res = ""
    length = len(conversation)
    for index, phrase in enumerate(conversation):
        res += (second if (length - index) % 2 else first) + phrase

    return res.strip()


def dict_to_dialog(d: dict[str, str]):
    global cols_ordered

    conv = []
    [conv.append(d[key]) for key in cols_ordered if d[key]]
    return {"dialog": add_tokens(conv)}

Проверяем, что все работает

In [ ]:
dict_to_dialog(
    {
        "context_3": "1",
        "context_2": "2!",
        "context_1": "3?",
        "response": "4)",
    }
)

Теперь можно замаппить данные в этот формат

In [ ]:
dialog_ds = ds.map(
    dict_to_dialog,
    remove_columns=cols_ordered + ["__index_level_0__"],
)

И токенизаривать

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-medium')

In [ ]:
def tokenize_text(text: dict[str, str]):
    return tokenizer(
        text["dialog"],
        max_length=1024,
        truncation=True,
        padding=True,
    )

In [ ]:
final_ds = dialog_ds.map(
    tokenize_text,
    remove_columns=["dialog"],
)
final_ds

## Fine-tuning

Будем `fine-tun`ить при помощи интерфейса `Trainer` с `huggingface`

Подключаем `cuda` если есть

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

Скачиваем и загружаем в память модель

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained('tinkoff-ai/ruDialoGPT-medium').to(device)

Настраиваем тренировку

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

*код для колаба, чтобы почистить память, <s>если модель упала</s>*

In [ ]:
# to clear GPU memory when training drops
with torch.no_grad():
    torch.cuda.empty_cache()

import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="output",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        max_steps=3000,
        eval_steps=100,
        save_steps=500,
        warmup_steps=10,
        save_total_limit=2,
    ),
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    ),
    train_dataset=final_ds["train"],
    eval_dataset=final_ds["test"],
)

<s>Ждем</s> Тренируем

In [ ]:
trainer.train()

Сохраняем, чтобы потом переиспользовать

In [ ]:
model.save_pretrained(model_output_path)

## Тесты!

In [ ]:
def add_tokens(conversation: list[str]):
    first, second = " @@ПЕРВЫЙ@@ ", " @@ВТОРОЙ@@ "
    if not conversation:
        return f"{second} "

    res = ""
    length = len(conversation)
    for index, phrase in enumerate(conversation):
        res += (first if (length - index) % 2 else second) + phrase

    return res.strip() + second

In [ ]:
add_tokens(
        [
            "Привет, ты кто?",
            "Я это я"
        ]
    ),

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_output_path)

In [ ]:
inputs = tokenizer(
    add_tokens(
        [
            "Привет, ты кто?",
            "Я это я"
        ]
    ),
    return_tensors="pt",
)

generated_token_ids = model.generate(
    **inputs,
    top_k=20,  # sample one of k most likely
    top_p=0.9,  # sample from those most likely which sum >= p
    num_beams=20,  # num beams for beam search
    num_return_sequences=3,  # how many candidates to return
    do_sample=True,  # do sample or greedy search
    no_repeat_ngram_size=2,  # n grams of this n must not repeat in a text
    temperature=1.5,  # make this value higher to get more interesting responses
    # repetition_penalty=2.0,  # make this value higher to fight with repetition
    length_penalty=0.5,  # < 1 for short texts, > 1 for long
    eos_token_id=50257,  # when to stop
    pad_token_id=tokenizer.eos_token_id,
    max_new_tokens=60,  # how many tokens to generate
)

print(*[
    tokenizer.decode(tokens)
    .split("@@ВТОРОЙ@@")[-1]
    .split("@@ПЕРВЫЙ@@")[0]
    .strip()
    for tokens in generated_token_ids
], sep="\n")

Вдоволь натестировав и изучив [все параметры \[3\]](https://huggingface.co/blog/how-to-generate), пришел к текущим параметрам для модели

## Ссылки

1. [tinkoff-ai github](https://github.com/tinkoff-ai/pycon-chit-chat)
2. [Link regex](https://stackoverflow.com/questions/3809401/what-is-a-good-regular-expression-to-match-a-url)
3. [Explanation by huggingface on how to generate using transofrmer model](https://huggingface.co/blog/how-to-generate)